In [90]:
import pandas as pd
import numpy as np

import statsmodels.regression.linear_model as lm

For a given function $u_n$ and a value $k$, $OP(k, n)$ is the best-fitting function. This is a matrix model question (regression). The model setup is 
$$
\begin{bmatrix} 1 & n & n^2 & \dots & n^{k-1} \end{bmatrix} \begin{bmatrix} a_0 \\ a_1 \\ a_2 \\ \vdots \\ a_{k-1} \end{bmatrix} \approx u_n.
$$

In particular, for $n \leq k$, we should have equality. So the best model is a projection of the form $(A^T A)^{-1} A^T u$ (see [OLS regression][1]) where
$$
A = \begin{bmatrix} 1 & 1 & 1^2 & \dots & 1^{k-1} \\ 1 & 2 & 2^2 & \dots & 2^{k-1} \\ 1 & 3 & 3^2 & \dots & 3^{k-1} \\ \vdots & \vdots & \ddots & \vdots \\ 1 & k & k^2 & \dots & k^{k-1} \\ \end{bmatrix}, \quad u = \begin{bmatrix} u_1 \\ u_2 \\ u_3 \\ \vdots \\ u_{k} \end{bmatrix}
$$
and the fitted model is
$$
\begin{bmatrix} 1 & 1 & 1^2 & \dots & 1^{k-1} \\ 1 & 2 & 2^2 & \dots & 2^{k-1} \\ 1 & 3 & 3^2 & \dots & 3^{k-1} \\ \vdots & \vdots & \ddots & \vdots \\ 1 & k & k^2 & \dots & k^{k-1} \\ \end{bmatrix} \begin{bmatrix} \hat{a}_0 \\ \hat{a}_1 \\ \hat{a}_2 \\ \vdots \\ \hat{a}_{k-1} \end{bmatrix} = \begin{bmatrix} \hat{u}_1 \\ \hat{u}_2 \\ \hat{u}_3 \\ \vdots \\ \hat{u}_{k} \end{bmatrix}.
$$

However, since $OLS$ uses floats (in matrix inverses), this method has error in practice. Thus we use [Newton interpolation][2]. [Lagrange interpolation][3] should work as well.

[1]: https://en.wikipedia.org/wiki/Ordinary_least_squares#Matrix/vector_formulation
[2]: https://en.wikipedia.org/wiki/Newton_polynomial
[3]: https://en.wikipedia.org/wiki/Lagrange_polynomial

In [115]:
# generate the x by x A matrix for OP(x, n)
def generate_sample_matrix(x):
    ret = np.ones((x,x), int)
    for i in range(x):
        for j in range(1,x):
            ret[i][j] = (i+1)*ret[i][j-1]

    return ret

In [409]:
# generate u_i, for each i up to x
def generate_solution(x):
    ret = np.zeros(x, int)
    for i in range(x):
        # ret[i] += (i+1)**3
        for n in range(11):
            ret[i] += (-1)**n * (i+1)**n

    return ret

In [410]:
# used OLS to solve, but was wrong because of floating point issues
def op_old(k, n):
    a = generate_sample_matrix(k)
    u = generate_solution(k)

    mod = lm.OLS(u, a)
    res = mod.fit() 
    coefs = np.array(np.round(res.params, 0), int)

    def apply_coefs(coefs, n):
        ret = np.zeros(n, int)
        for i in range(n):
            for x in range(len(coefs)):
                ret[i] += coefs[x]*(i+1)**x
        
        return np.array(np.round(ret, 0), int)
    
    return coefs, apply_coefs(coefs, n)


In [411]:
# implement divided differences (forward differencing) for newton's coefficients
def divided_differences(xs, ys):
    if len(xs) != len(ys):
        raise ValueError('Mismatching data points: number of xs needs to match number of ys')
    
    k = len(ys)
    coefs = [ys[0]]
    differences = ys.copy()

    for i in range(1, k):
        new_differences = []
        for j in range(1, len(differences)):
            new_differences.append((differences[j] - differences[j-1])/(xs[j+i-1] - xs[j-1]))
        
        differences = new_differences.copy()
        coefs.append(differences[0])
    
    return np.array(coefs, int)
        

In [421]:
# interpolate based on k points, and plug in n
def newton_polynomial(xs, ys, n, coefs = None):
    if coefs is None:
        coefs = divided_differences(xs, ys).copy()
    s = coefs[0]
    for i in range(1, len(coefs)):
        prod = 1
        for j in range(i):
            prod *= n - xs[j]

        s += coefs[i]*prod

    return s

In [439]:
s = 0
u = generate_solution(12)
for k in range(1, 12):
    xs = [i+1 for i in range(k)]
    ys = generate_solution(k)
    npcs = divided_differences(xs, ys)
    for n in range(1, k+2):    
        print(newton_polynomial(xs, ys, n, coefs = npcs), u[n-1])

    print()
    if k == 11:
        break
    s += newton_polynomial(xs, ys, k+1, coefs = npcs)

print(s)

1 1
1 683

1 1
683 683
1365 44287

1 1
683 683
44287 44287
130813 838861

1 1
683 683
44287 44287
838861 838861
3092453 8138021

1 1
683 683
44287 44287
838861 838861
8138021 8138021
32740951 51828151

1 1
683 683
44287 44287
838861 838861
8138021 8138021
51828151 51828151
205015603 247165843

1 1
683 683
44287 44287
838861 838861
8138021 8138021
51828151 51828151
247165843 247165843
898165577 954437177

1 1
683 683
44287 44287
838861 838861
8138021 8138021
51828151 51828151
247165843 247165843
954437177 954437177
3093310441 3138105961

1 1
683 683
44287 44287
838861 838861
8138021 8138021
51828151 51828151
247165843 247165843
954437177 954437177
3138105961 3138105961
9071313571 9090909091

1 1
683 683
44287 44287
838861 838861
8138021 8138021
51828151 51828151
247165843 247165843
954437177 954437177
3138105961 3138105961
9090909091 9090909091
23772343751 23775972551

1 1
683 683
44287 44287
838861 838861
8138021 8138021
51828151 51828151
247165843 247165843
954437177 954437177
3138105